In [1]:
import numpy as np
import matplotlib.pylab as plt
import random
from scipy.special import comb

import sys
sys.path.append('../src/')
sys.path.append('../scripts/')

In [2]:
import numpy as np
import networkx as nx
import random
from itertools import combinations

In [3]:
from generators_sc_test import regular_maximum_overlapped_simplicial_complex

In [4]:
n = 200
k = 3
N = n * k
print(f"N = n * k = {N}")
print(f"N % 2 == 0: {N % 2 == 0}")

N = n * k = 600
N % 2 == 0: True


In [5]:
N_realized, np_edges, np_triangles = regular_maximum_overlapped_simplicial_complex(n, k)
N_realized, len(np_edges), len(np_triangles)

(600, 900, 200)

In [6]:
edges = set([(int(u), int(v)) for (u, v) in list(np_edges)])
triangles = set([(int(u), int(v), int(w)) for (u, v, w) in list(np_triangles)])
len(edges), len(triangles)

(900, 200)